In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import g_key

In [2]:
# Import cities file into the cities_pd DataFrame
cities_df = pd.read_csv("Cities.csv")

In [3]:
# Add columns for lat, lng, airport name, airport address, airport rating.
# Hint: Be sure to specify "" as the initial value for each column.
cities_df['lat'] = ""
cities_df['lng'] = ""
cities_df['airport_name'] = ""
cities_df['airport_address'] = ""
cities_df['airport_rating'] = ""

In [4]:
# Loop through the cities_pd and get the lat/long for each city
# Hint: `requests.get(target_url).json()`
# Hint: `for index, row in cities_pd.iterrows():`
# Hint: `cities_pd.loc`
for index, row in cities_df.iterrows():
    target_city = row['City']
    geocode_params = {"address": target_city, "key": g_key}

    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"

    locations = requests.get(geocode_url, params=geocode_params).json()

    cities_df.loc[index, 'lat'] = locations["results"][0]["geometry"]["location"]["lat"]
    cities_df.loc[index, 'lng'] = locations["results"][0]["geometry"]["location"]["lng"]

cities_df.head()

,City,State,lat,lng,airport_name,airport_address,airport_rating
0,New York City,New York,40.7128,-74.006,,,
1,Los Angeles,California,34.0522,-118.244,,,
2,Chicago,Illinois,41.8781,-87.6298,,,
3,Houston,Texas,29.7604,-95.3698,,,
4,Philadelphia,Pennsylvania,39.9526,-75.1652,,,


In [5]:
# Use the lat/lng we recovered to identify airports
# Hint: Code is similar to above block (i.e. you will need to use cities_pd.iterrows() again)
# Hint: Use the keyword search for "international+airport"
# Hint: Use a try-except to handle airports that are missing ratings

# use iterrows to iterate through pandas dataframe
for index, row in cities_df.iterrows():

    lat = row['lat'] 
    lng = row['lng']

    # get lat, lng from rows
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
    "radius": 50000,
    "types": "airport",
    "keyword": "international airport",
    "key": g_key, 
    "location": f'{lat},{lng}'
}

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    cities_df.loc[index, 'airport_name'] = results[0]["name"]
    cities_df.loc[index, 'airport_address'] = results[0]["vicinity"]
    cities_df.loc[index, 'airport_rating'] = results[0]["rating"]
    # extract results


In [6]:
cities_df.head()

,City,State,lat,lng,airport_name,airport_address,airport_rating
0,New York City,New York,40.7128,-74.006,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.1
1,Los Angeles,California,34.0522,-118.244,Los Angeles International Airport,"1 World Way, Los Angeles",3.6
2,Chicago,Illinois,41.8781,-87.6298,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.6
3,Houston,Texas,29.7604,-95.3698,Grupo Tampico International,"7930 Airport Blvd, Houston",0
4,Philadelphia,Pennsylvania,39.9526,-75.1652,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.3


In [10]:
import gmaps
import gmaps.datasets

# Google developer API key
from config import g_key
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    cities_df[['lat', 'lng']], weights=cities_df['airport_rating'], dissipating=False,
    max_intensity=30, point_radius=3.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))